# Екзамен Дровольського Ярослава, ІПС-42
## (частина 2)

Імпортуємо бібліотеку Pandas

In [1]:
import pandas as pd

Далі було завантажено набір даних про оцінки студентів на іспитах за [посиланням](https://www.kaggle.com/datasets/desalegngeb/students-exam-scores)

Читаємо CSV-файл у DataFrame:

In [3]:
df = pd.read_csv('Expanded_data_with_more_features.csv')

Подивимося на загальний вигляд таблиці.

In [4]:
df

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,816,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65
30637,890,male,group E,high school,standard,none,single,regularly,no,1.0,private,5 - 10,58,53,51
30638,911,female,NaN,high school,free/reduced,completed,married,sometimes,no,1.0,private,5 - 10,61,70,67
30639,934,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,5 - 10,82,90,93


## Задача класифікації 5 балів
Передбачте, чи завершив студент курс підготовки до іспиту, на основі його демографічних та академічних даних.
   
**Вхідні дані:** Gender, EthnicGroup, ParentEduc, LunchType, ParentMaritalStatus, PracticeSport, IsFirstChild, NrSiblings, TransportMeans, WklyStudyHours, MathScore, ReadingScore, WritingScore

**Результат:** TestPrep

### 1. Дослідження атрибутів датасету


Оскільки датасет містить рядки, де значення деяких атрибутів не вказані, то ми приберемо з датасету такі рядки і далі будемо працювати без них:

In [10]:
dfc = df.dropna()

Переглянемо інформацію про тип даних атрибутів

In [11]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19243 entries, 2 to 30640
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           19243 non-null  int64  
 1   Gender               19243 non-null  object 
 2   EthnicGroup          19243 non-null  object 
 3   ParentEduc           19243 non-null  object 
 4   LunchType            19243 non-null  object 
 5   TestPrep             19243 non-null  object 
 6   ParentMaritalStatus  19243 non-null  object 
 7   PracticeSport        19243 non-null  object 
 8   IsFirstChild         19243 non-null  object 
 9   NrSiblings           19243 non-null  float64
 10  TransportMeans       19243 non-null  object 
 11  WklyStudyHours       19243 non-null  object 
 12  MathScore            19243 non-null  int64  
 13  ReadingScore         19243 non-null  int64  
 14  WritingScore         19243 non-null  int64  
dtypes: float64(1), int64(4), object(10)
memor

Бачимо, що всього 15 атрибутів: **10 рядкових**, **5 числових**

Для кожного нечислового атрибуту дивимося множину його значень:

In [12]:
print(dfc["Gender"].value_counts(), "\n")

print(dfc["EthnicGroup"].value_counts(), "\n")

print(dfc["ParentEduc"].value_counts(), "\n")

print(dfc["LunchType"].value_counts(), "\n")

print(dfc["TestPrep"].value_counts(), "\n")

print(dfc["ParentMaritalStatus"].value_counts(), "\n")

print(dfc["PracticeSport"].value_counts(), "\n")

print(dfc["IsFirstChild"].value_counts(), "\n")

print(dfc["TransportMeans"].value_counts(), "\n")

print(dfc["WklyStudyHours"].value_counts(), "\n")

Gender
female    9775
male      9468
Name: count, dtype: int64 

EthnicGroup
group C    6181
group D    4970
group B    3915
group E    2712
group A    1465
Name: count, dtype: int64 

ParentEduc
some college          4425
high school           3773
associate's degree    3749
some high school      3658
bachelor's degree     2293
master's degree       1345
Name: count, dtype: int64 

LunchType
standard        12472
free/reduced     6771
Name: count, dtype: int64 

TestPrep
none         12587
completed     6656
Name: count, dtype: int64 

ParentMaritalStatus
married     11009
single       4608
divorced     3256
widowed       370
Name: count, dtype: int64 

PracticeSport
sometimes    9715
regularly    6950
never        2578
Name: count, dtype: int64 

IsFirstChild
yes    12417
no      6826
Name: count, dtype: int64 

TransportMeans
school_bus    11280
private        7963
Name: count, dtype: int64 

WklyStudyHours
5 - 10    10499
< 5        5381
> 10       3363
Name: count, dtype: int64 



### 2. Створення тестового та тренувального наборів даних

Відділяємо в датасеті предиктори (predictors, `X`) та мітки (labels, `y`):

In [14]:
X = dfc.drop("TestPrep", axis=1) # predictors
y = dfc["TestPrep"].copy() # labels

Створюємо тестовий та тренувальний набори:

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

### 3. Підготовка даних

Створимо **пайплайн для числових атрибутів**:
1. Хоч пропущених даних немає, але включимо в пайпалйн **SimpleImputer** з медіаною на випадок, якщо колись будуть дані з пропусками
2. Засосовуємо Feature scaling

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# pipeline for numerical attributes
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler())
])

Створимо **pipeline**, який буде трансформувати **всі атрибути**: і числові і категорійні

Тут вилучаємо атрибут `Unnamed: 0`, оскільки він відповідає за нумерацію в датасеті і жодним чином не впливає на класифікацію.

Рядкові (категорійні) атрибути `ParentEduc`, `PracticeSport` та `WklyStudyHours` перетворюємо на числа відповідно до логічного порядку по кращості. Кожен інший категорійний атрибут перетворюємо на бінарні атрибути (застосовуємо one-hot encoding).

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


num_attribs = ['NrSiblings', 'MathScore', 'ReadingScore', 'WritingScore']
cat_ord_attribs = ['ParentEduc', 'PracticeSport', 'WklyStudyHours']
cat_noord_attribs = ['Gender', 'EthnicGroup', 'LunchType', 'ParentMaritalStatus', 'IsFirstChild', 'TransportMeans']

full_pipeline = ColumnTransformer([
        ('column_dropper', 'drop', ["Unnamed: 0"]),
        ('num', num_pipeline, num_attribs),
        ("parent_education_tr", OrdinalEncoder(categories=[["some high school", 'high school', 
                                                            'some college', "bachelor's degree", "master's degree",
                                                            "associate's degree"]]), ["ParentEduc"]),
        ("practice_sport_tr", OrdinalEncoder(categories=[['never', 'sometimes', 'regularly']]), ["PracticeSport"]),
        ("wkly_study_hours_tr", OrdinalEncoder(categories=[['< 5', '5 - 10', '> 10']]), ["WklyStudyHours"]),
        ("color_tr", OneHotEncoder(), cat_noord_attribs)
])

X_train_prepared = full_pipeline.fit_transform(X_train)

Підготуємо мітки (`y`) (перетворюємо категорії в числа):

In [30]:
# y is vector of labels
def prepare_labels(y): # NOT USED
    d = {x: i for i, x in enumerate(set(y))}
    return [d[x] for x in y]

In [36]:
# y_train_prepared = (y_train == 'completed')
y_train_prepared = prepare_labels(y_train)

### 4. Тренування класифікатора

In [37]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train_prepared, y_train_prepared)

SGDClassifier(random_state=42)

Визначаємо найкращі параметри:

In [68]:
from sklearn.model_selection import GridSearchCV

params = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 5], 
    'power_t': [0, 0.005, 0.1, 0.5, 1, 5, 10]
}
grid_search_cv = GridSearchCV(SGDClassifier(random_state=42), params, verbose=1, cv=3)

grid_search_cv.fit(X_train_prepared, y_train_prepared)

Fitting 3 folds for each of 42 candidates, totalling 126 fits


GridSearchCV(cv=3, estimator=SGDClassifier(random_state=42),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 5],
                         'power_t': [0, 0.005, 0.1, 0.5, 1, 5, 10]},
             verbose=1)

In [69]:
grid_search_cv.best_params_

{'alpha': 0.001, 'power_t': 0}

Виміряємо **точність** класифікатора за допомогою крос-валідації **на тренувальних даних**:

In [70]:
from sklearn.model_selection import cross_val_score

cross_val_score(grid_search_cv.best_estimator_, X_train_prepared, y_train_prepared, cv=10, scoring="accuracy").mean()

0.7471573053983025

Виміряємо **точність** класифікатора за допомогою крос-валідації на **тестових даних**:

In [71]:
X_test_prepared = full_pipeline.fit_transform(X_test)
y_test_prepared = prepare_labels(y_test)

cross_val_score(grid_search_cv.best_estimator_, X_test_prepared, y_test_prepared, cv=10, scoring="accuracy").mean()

0.7281230320649408